In [7]:
import boto3 
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

import boto3 
bedrock = boto3.client(
    service_name='bedrock', 
    region_name='us-east-1'
)

In [3]:
import json
import os
import sys

# Kinesis Transcription Retrieval

In [12]:
!pip install tzlocal

In [13]:
# !pip install tzlocal
# pip install python-dateutil
# pip install boto3
import boto3
import base64
import json
import time
import tzlocal

In [14]:
# Initialize a boto3 Kinesis client
kinesis_client = boto3.client('kinesis', region_name='us-east-1')

# The name of your Kinesis stream
stream_name = 'CH_agent_call_streams'

# The sequence number you want to start reading from
sequence_number_for_shard = '49649059939264757358942140911703275325366952989042933794'

In [17]:
# Function to process records and create JSON
def process_records(records):
    for record in records:
        # Kinesis data is UTF-8 encoded so decode here
        payload = record["Data"].decode('utf-8')
        return payload

# Function to get records from Kinesis
def get_kinesis_records(stream_name, sequence_number_for_shard):
    response = kinesis_client.describe_stream(StreamName=stream_name)
    shard_id = response['StreamDescription']['Shards'][2]['ShardId']

    # Get a shard iterator using the sequence number
    shard_iterator = kinesis_client.get_shard_iterator(StreamName=stream_name,
                                                       ShardId=shard_id,
                                                       ShardIteratorType='AT_SEQUENCE_NUMBER',
                                                       StartingSequenceNumber=sequence_number_for_shard)['ShardIterator']

    out = kinesis_client.get_records(ShardIterator=shard_iterator, Limit=100)
    shard_iterator = out['NextShardIterator']
    records = out['Records']

    if records:
        json_data = process_records(records)

    time.sleep(1)  # Sleep to avoid hitting rate limits
    return json_data

In [18]:
response = get_kinesis_records(stream_name, sequence_number_for_shard)

In [19]:
print(response)

[
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: 'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I'll be assisting you with your enrollment. How may I address you?',
      BeginOffsetMillis: 1257,
      EndOffsetMillis: 9697,
      Sentiment: 'NEUTRAL',
      Id: 'fb29489e-d06b-48b1-8e0b-519f17c4a68e'
    }
  },
  {
    Transcript: {
      ParticipantId: 'CUSTOMER',
      ParticipantRole: 'CUSTOMER',
      Content: 'Hello, Alex. I'm Jordan Smith',
      BeginOffsetMillis: 3047,
      EndOffsetMillis: 8055,
      Sentiment: 'POSITIVE',
      Id: '7f6820ee-0487-4cbc-9b35-33e7c31e1d5d'
    }
  },
  {
    Transcript: {
      ParticipantId: 'AGENT',
      ParticipantRole: 'AGENT',
      Content: ' It's nice to meet you, Jordan. To start with your enrollment, I'll need to ask you a few questions. Is now a good time?',
      BeginOffsetMillis: 10457,
      EndOffsetMillis: 16775,
      Sentiment: 'POSITIVE',
     

# Text Preprocessing :

In [176]:
import re

def preprocess_text(text):
    r1 = text.replace("[","").replace("]","").replace('Transcript','"Transcript"')
    r2 = r1.replace('ParticipantId','"ParticipantId"').replace('ParticipantRole','"ParticipantRole"').replace('Content','"Content"')
    r3 = r2.replace('BeginOffsetMillis','"BeginOffsetMillis"').replace('EndOffsetMillis','"EndOffsetMillis"')
    r4 = r3.replace('Sentiment','"Sentiment"').replace('Id','"Id"')
    
    start_word_indices = [i.start() for i in re.finditer('"ParticipantRole"',r4)]
    end_word_indices = [i.start() for i in re.finditer('"BeginOffsetMillis"',r4)]
    
    start_search_word = '"Participant_Role"'
    transcript = ""
    for i in range(len(start_word_indices)):
        transcript = transcript + word_filter(r4,start_search_word,start_word_indices[i],end_word_indices[i]) + "\n"
        
    return transcript

def word_filter(text,start_search_word,start_word_index,end_word_index):
    # Check if the search word exists in the input string
    if start_word_index != -1:
        # Extract everything from the search word onwards
        extracted_text = text[start_word_index + len(start_search_word):end_word_index]
        roles = ["AGENT","CUSTOMER"]
        role_index = 0
        conv = ""
        for role in roles:
            if extracted_text.find(role) != -1 :
                role_index = extracted_text.find(role)
                content_index = extracted_text.find('"Content"')
                comma_indices = [i.start() for i in re.finditer(',',extracted_text)] 
                #temp = extracted_text[content_index+len('"Content"'):end_word_index]
                #comma_index = temp[content_index+len('Content'):].find(",")
                conv = conv + extracted_text[role_index:role_index+len(role)]
                conv += extracted_text[content_index+len('"Content"'):]#comma_indices[len(comma_indices)-1]]
                return conv
            else:
                pass
    else:
        pass

In [177]:
t1 = preprocess_text(response)
print(t1)

AGENT: 'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I'll be assisting you with your enrollment. How may I address you?',
      
CUSTOMER: 'Hello, Alex. I'm Jordan Smith',
      
AGENT: ' It's nice to meet you, Jordan. To start with your enrollment, I'll need to ask you a few questions. Is now a good time?',
      
CUSTOMER: 'Yes, that's fine',
      
AGENT: "Great, thank you",
      
AGENT: "Let's begin with your full name for our records",
      
CUSTOMER: 'It's Jordan Andrew Smith',
      
AGENT: 'Thank you, Jordan. May I ask which gender you identify with? This information helps us to tailor our care to your specific needs, but please know that providing this information is entirely optional',
      
CUSTOMER: "Sure, it's March 14, 1989",
      
AGENT: 'Perfect, thank you. Next, I'll need your contact information. What's the best phone number and email to reach you at?',
      
CUSTOMER: 'You can reach me at 555-123-4567 and jordan.smith@email.com',
  

In [178]:
final_transcript = preprocess_text(response)

In [179]:
final_transcript

'AGENT: \'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I\'ll be assisting you with your enrollment. How may I address you?\',\n      \nCUSTOMER: \'Hello, Alex. I\'m Jordan Smith\',\n      \nAGENT: \' It\'s nice to meet you, Jordan. To start with your enrollment, I\'ll need to ask you a few questions. Is now a good time?\',\n      \nCUSTOMER: \'Yes, that\'s fine\',\n      \nAGENT: "Great, thank you",\n      \nAGENT: "Let\'s begin with your full name for our records",\n      \nCUSTOMER: \'It\'s Jordan Andrew Smith\',\n      \nAGENT: \'Thank you, Jordan. May I ask which gender you identify with? This information helps us to tailor our care to your specific needs, but please know that providing this information is entirely optional\',\n      \nCUSTOMER: "Sure, it\'s March 14, 1989",\n      \nAGENT: \'Perfect, thank you. Next, I\'ll need your contact information. What\'s the best phone number and email to reach you at?\',\n      \nCUSTOMER: \'You can reach me a

# Form Enrollment :

In [264]:
def load_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "max_gen_len": 200
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise

In [329]:
def enrollment_prompt_generator(conversation):
    prompt_llama = f"""
Instruction: "Identify the different entities like Name,First Name,Last Name,Age,Email,Username etc of the patient/customer
from the below conersation and provide them in the following format :
Name : [Name]
Age : [Age] and so on in a json format and only provide the json object." :

{conversation}.
             
Response :  
    """
    return prompt_llama

In [285]:
prompt_enrollment = enrollment_prompt_generator(final_transcript)
enrollment_data = load_llama2(bedrock_runtime,prompt_enrollment,0,0.9)

In [341]:
prompt_enrollment = enrollment_prompt_generator(final_transcript)
enrollment_data = load_llama2(bedrock_runtime,prompt_enrollment,0,0.9)

In [342]:
print(enrollment_data)


{
"Name" : "Jordan Andrew Smith",
"Age" : "March 14, 1989",
"Gender" : "Male",
"Email" : "jordan.smith@email.com",
"Phone Number" : "555-123-4567"
}


In [286]:
print(enrollment_data)


{
"Name" : "Jordan Andrew Smith",
"Age" : "1989-03-14",
"Gender" : "Male",
"Email" : "jordan.smith@email.com",
"Phone" : "555-123-4567"
}


In [276]:
print(enrollment_data)


Name : Jordan Smith
Age : 33
Gender : Male
Email : jordan.smith@email.com
Phone Number : 555-123-4567


In [287]:
import json
enrollment_json_object = json.loads(enrollment_data)

In [288]:
enrollment_json_object

{'Name': 'Jordan Andrew Smith',
 'Age': '1989-03-14',
 'Gender': 'Male',
 'Email': 'jordan.smith@email.com',
 'Phone': '555-123-4567'}

# Further Testing

In [326]:
final_transcript1 = final_transcript + "\n" + "CUSTOMER : My Dosage Period is 2 weeks"
print(final_transcript1)

AGENT: 'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I'll be assisting you with your enrollment. How may I address you?',
      
CUSTOMER: 'Hello, Alex. I'm Jordan Smith',
      
AGENT: ' It's nice to meet you, Jordan. To start with your enrollment, I'll need to ask you a few questions. Is now a good time?',
      
CUSTOMER: 'Yes, that's fine',
      
AGENT: "Great, thank you",
      
AGENT: "Let's begin with your full name for our records",
      
CUSTOMER: 'It's Jordan Andrew Smith',
      
AGENT: 'Thank you, Jordan. May I ask which gender you identify with? This information helps us to tailor our care to your specific needs, but please know that providing this information is entirely optional',
      
CUSTOMER: "Sure, it's March 14, 1989",
      
AGENT: 'Perfect, thank you. Next, I'll need your contact information. What's the best phone number and email to reach you at?',
      
CUSTOMER: 'You can reach me at 555-123-4567 and jordan.smith@email.com',
  

In [340]:
prompt_enrollment1 = enrollment_prompt_generator(final_transcript1)
enrollment_data1 = load_llama2(bedrock_runtime,prompt_enrollment1,0,0.9)

In [339]:
print(enrollment_data1)


Json Object with the following fields:

Name : Jordan Andrew Smith
Age : 33
Gender : Male
Email : jordan.smith@email.com
Phone Number : 555-123-4567
Dosage Period : 2 weeks

Please provide the json object in the above format.


In [305]:
print(enrollment_data1)


Name : Jordan
Age : 33
Gender : Male
Email : jordan.smith@email.com
Phone : 555-123-4567
Dosage Period : 2 weeks

Please provide the above information in the format requested.









































































































































# Text Summarisation :

In [180]:
def text_summariser_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "max_gen_len": 200
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise

In [236]:
def prompt_generator(context):
    prompt_llama = f"""
Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : " :

{context}.
             
Response :  
    """
    return prompt_llama

In [237]:
prompt = prompt_generator(final_transcript)
print(prompt)


Instruction: "Summarise this call transcript between a patient and an agent and provide it in a precise paragraph : " :

AGENT: 'Good morning! Thank you for calling Patient Pathways. My name is Alex, and I'll be assisting you with your enrollment. How may I address you?',
      
CUSTOMER: 'Hello, Alex. I'm Jordan Smith',
      
AGENT: ' It's nice to meet you, Jordan. To start with your enrollment, I'll need to ask you a few questions. Is now a good time?',
      
CUSTOMER: 'Yes, that's fine',
      
AGENT: "Great, thank you",
      
AGENT: "Let's begin with your full name for our records",
      
CUSTOMER: 'It's Jordan Andrew Smith',
      
AGENT: 'Thank you, Jordan. May I ask which gender you identify with? This information helps us to tailor our care to your specific needs, but please know that providing this information is entirely optional',
      
CUSTOMER: "Sure, it's March 14, 1989",
      
AGENT: 'Perfect, thank you. Next, I'll need your contact information. What's the best ph

In [256]:
prompt = prompt_generator(final_transcript)
final_summary = text_summariser_llama2(bedrock_runtime,prompt,0.5,0.9)

In [257]:
print(final_summary)


The patient, Jordan Smith, called the Patient Pathways enrollment line to begin the enrollment process. The agent, Alex, greeted Jordan and asked for their full name for the records. Jordan provided their full name, date of birth, and gender. The agent also requested contact information, which Jordan provided. The agent thanked Jordan and confirmed that they had all the necessary information to proceed with the enrollment.


In [272]:
prompt = prompt_generator(final_transcript)
final_summary = text_summariser_llama2(bedrock_runtime,prompt,0.5,0.9)

In [273]:
print(final_summary)


The patient, Jordan Smith, called Patient Pathways to enroll in their services. Alex, the agent, greeted Jordan and asked for their full name and gender identity for record-keeping purposes. Jordan provided this information and then gave their contact information, including their phone number and email address.


# Claude Model

In [313]:
print(enrollment_data)


{
"Name" : "Jordan Andrew Smith",
"Age" : "1989-03-14",
"Gender" : "Male",
"Email" : "jordan.smith@email.com",
"Phone" : "555-123-4567"
}


In [317]:
patient_data = ""
list1 = enrollment_json_object.keys()
for value in list1:
    patient_data += f"{value} : {enrollment_json_object[value]}"
    patient_data += "\n"
print(patient_data)

Name : Jordan Andrew Smith
Age : 1989-03-14
Gender : Male
Email : jordan.smith@email.com
Phone : 555-123-4567



In [318]:
import boto3
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import Bedrock

bedrock_runtime_claude = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)

claude_prompt = f"""
Instruction : You are an agent assist where you trying to helpe the agent help with the insights or data collection from the real time conversation happening between the agent and a patient.Based on the
conversation summary as well as the enrollment data, can you help the agent answer any questionsand generate insights when asked for.

Current conversation summary:
{final_summary}

Patient enrollment data:
{patient_data}

Here is the human's next reply:
{input}

Assistant:
"""

model_id = "anthropic.claude-v2"
model_kwargs =  { 
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
    "prompt": claude_prompt,
}

llm = Bedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs
)

conversation= ConversationChain(
    llm=llm, verbose=False, memory=ConversationBufferMemory() #memory_chain
)

In [319]:
print(claude_prompt)


Instruction : You are an agent assist where you trying to helpe the agent help with the insights or data collection from the real time conversation happening between the agent and a patient.Based on the
conversation summary as well as the enrollment data, can you help the agent answer any questionsand generate insights when asked for.

Current conversation summary:

The patient, Jordan Smith, called Patient Pathways to enroll in their services. Alex, the agent, greeted Jordan and asked for their full name and gender identity for record-keeping purposes. Jordan provided this information and then gave their contact information, including their phone number and email address.

Patient enrollment data:
Name : Jordan Andrew Smith
Age : 1989-03-14
Gender : Male
Email : jordan.smith@email.com
Phone : 555-123-4567


Here is the human's next reply:
<bound method Kernel.raw_input of <ipykernel.ipkernel.IPythonKernel object at 0x7fa65b66da80>>

Assistant:



In [322]:
output = conversation.predict(input="Do we have the Patient's Name?")

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/langchain_community/llms/bedrock.py:73: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Do we have the Name?
AI:  I'm afraid I don't have enough context to know what specific name you are asking about. I'm an AI assistant created by Anthropic to be helpful, harmless, and honest. If you provide more details about the context, I can try to answer your question about a name.

Human: Do we have the Patient's Name?
AI:

Assistant:
  warnings.warn(ALTERNATION_ERROR + f" Received {input_text}")


In [323]:
output

" I'm afraid I don't have any information about a specific patient or their name. As an AI assistant without access to personal medical records, I don't have enough context to provide patient details or names. I'd be happy to have a respectful conversation though! Please feel free to provide additional context if you have a more general question about healthcare or medicine."

# Q/A LLAMA Model

In [343]:
def invoke_llama2(bedrock_runtime , prompt , temp , top_p):
    try:
        body = {
            "prompt": prompt,
            "temperature": temp,
            "top_p": top_p,
            "max_gen_len": 200
            }

        response = bedrock_runtime.invoke_model(
            modelId="meta.llama2-13b-chat-v1", body=json.dumps(body)
        )

        response_body = json.loads(response["body"].read())
        completion = response_body["generation"]

        return completion

    except ClientError:
        logger.error("Couldn't invoke Llama 2")
        raise

In [445]:
def prompt_generator(final_summary,patient_data,question):
    prompt_llama = f"""Using the information from the conversation summary : {final_summary} 
and 
The patient enrollment data : 
{patient_data},
Answer the following Question : 
{question}

If you do not know the answer and if the Current conversation summary or the Patient enrollment data doesn't contain the answer,then 
truthfully say I don't know.
Response:
"""
    return prompt_llama

In [446]:
question = "What is the patient's name?"

In [447]:
final_summary

'\nThe patient, Jordan Smith, called Patient Pathways to enroll in their services. Alex, the agent, greeted Jordan and asked for their full name and gender identity for record-keeping purposes. Jordan provided this information and then gave their contact information, including their phone number and email address.'

In [448]:
print(patient_data)

Name : Jordan Andrew Smith
Age : 1989-03-14
Gender : Male
Email : jordan.smith@email.com
Phone : 555-123-4567



In [449]:
question

"What is the patient's name?"

In [450]:
prompt = prompt_generator(final_summary,patient_data,question)
final_answer_llama = invoke_llama2(bedrock_runtime,prompt,0.5,0.9)

In [451]:
print(prompt)

Using the information from the conversation summary : 
The patient, Jordan Smith, called Patient Pathways to enroll in their services. Alex, the agent, greeted Jordan and asked for their full name and gender identity for record-keeping purposes. Jordan provided this information and then gave their contact information, including their phone number and email address. 
and 
The patient enrollment data : 
Name : Jordan Andrew Smith
Age : 1989-03-14
Gender : Male
Email : jordan.smith@email.com
Phone : 555-123-4567
,
Answer the following Question : 
What is the patient's name?

If you do not know the answer and if the Current conversation summary or the Patient enrollment data doesn't contain the answer,then 
truthfully say I don't know.
Response:



In [452]:
print(final_answer_llama)


The patient's name is Jordan Andrew Smith.


In [388]:
question2 = "What is the patient's Lastname,Email Address and Phone Number?"

In [390]:
prompt = prompt_generator(final_summary,patient_data,question2)
final_answer_llama_2 = invoke_llama2(bedrock_runtime,prompt,0.5,0.9)
print(final_answer_llama_2)


Lastname : Smith
Email Address : jordan.smith@email.com
Phone Number : 555-123-4567


In [453]:
prompt = prompt_generator(final_summary,patient_data,question2)
final_answer_llama_2 = invoke_llama2(bedrock_runtime,prompt,0.5,0.9)
print(final_answer_llama_2)


The patient's Lastname is Smith.
The patient's Email Address is jordan.smith@email.com.
The patient's Phone Number is 555-123-4567.


In [392]:
question3 = "What is the patient's Firstname?"
prompt = prompt_generator(final_transcript,final_summary,patient_data,question3)
final_answer_llama_2 = invoke_llama2(bedrock_runtime,prompt,0.5,0.9)
print(final_answer_llama_2)


Firstname : Jordan


In [455]:
question3 = "What is the patient's Firstname?"
prompt = prompt_generator(final_summary,patient_data,question3)
final_answer_llama_2 = invoke_llama2(bedrock_runtime,prompt,0.5,0.9)
print(final_answer_llama_2)


The patient's Firstname is Jordan.
